# Testing the McGurk effect on Machine Learning models

### Defining the McGurk effect experiments

In [ ]:
from experiments import McGurkExperiment

# Instantiate the list of experiments
experiments = [
    McGurkExperiment("ba", "ga", "da"), # ba (auditory) + ga (visual) = da  (fusioned sound)
    #McGurkExperiment("ba", "fa", "va"), # ba (auditory) + fa (visual) = va  (fusioned sound)
    #McGurkExperiment("ga", "ba", "bga") # ga (auditory) + ba (visual) = bga (combined sound)
]
masked_experiment = False

### Ensuring reproducible experiment results

In [ ]:
# Set the seeds for the experiments to ensure reproducible results
import torch
torch.manual_seed(42)
import random
random.seed(0)
import numpy as np
np.random.seed(0)

## Testing the effect on pretrained PerceiverIO models with regression mapping

### Training the models

In [ ]:
from models import McGurkPerceiver

perceiver_models = []
for experiment in experiments:
    # Instantiate a Perceiver model for the given experiment
    model = McGurkPerceiver(experiment)
    perceiver_models.append(model)

for model in perceiver_models:
    print(model.name())
    # Train the models
    _, _, _, _ = model.train(epochs=40000, learning_rate=0.002, train_with_masks=masked_experiment)

### Testing the model on normal samples

In [ ]:
testing_results = []
for model in perceiver_models:
    print(model.name())
    if masked_experiment:
        average_a, average_v, average_av = model.test(test_with_masks=masked_experiment)
        print(f"A TESTS : {auditory_tests}")
        print(f"V TESTS : {visual_tests}")
        print(f"MG TESTS : {mcgurk_tests}")
        testing_results.append((average_a, average_v, average_av))
    else:
        correct, mcgurk = model.test(test_with_masks=masked_experiment)
        testing_results.append((float(correct), float(mcgurk)))

print(testing_results)

### Generating the McGurk predictions

In [ ]:
model_predictions_avg = []

# Test the models on McGurk effect videos
for model in perceiver_models:
    print(model.name())
    predictions = model.test_mcgurk()
    # Average the predictions\n",
    predictions_avg = predictions.cpu().detach().numpy().mean(axis=0)
    print(predictions_avg)
    model_predictions_avg.append(predictions_avg)

# Convert the list to a numpy array
model_predictions_avg = np.array(model_predictions_avg)

### Results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def plot_mcgurk_confidences():
    # Plot the average confidence scores for each sound of each experiment
    fig, ax = plt.subplots(figsize=(6, 4), constrained_layout=True)

    # Define the axises labels and plot title
    #ax.set_title('Average confidence scores per syllable for each experiment')
    ax.set_xlabel('Experiment')
    ax.set_ylabel('Average confidence score per syllable')

    # Define bars for each type of syllables
    ind = np.arange(len(experiments))  # the x locations for the groups
    width = 0.3
    plt.bar(ind - width, model_predictions_avg[:,0], width, label='Auditory syllable')
    plt.bar(ind, model_predictions_avg[:,1], width, label='Visual syllable')
    plt.bar(ind + width, model_predictions_avg[:,2], width, label='McGurk effect syllable')

    plt.xticks(ind + width / 2, (experiments[0].to_str(), experiments[1].to_str(), experiments[2].to_str()))
    plt.legend(bbox_to_anchor=(1.1, 0.5), loc='center left')
    plt.yscale('log')
    plt.show()

def plot_test_confidences():
    # certifies the model is potent at recognizing syllables
    ...

def plot_mgurk_confidence_increase():
    # maybe, only if it's information rich
    ...



# TODO: Plot the average confidence scores for each normal sample of each experiment -> If possible, with test set
# TODO: Plot also for McGurk samples
# also maybe TODO: plot the confidence increase from normal samples to mcgurk samples for the mcgurk syllable (if it's interesting) -> maybe on a logscale
# and #TODO at home with PC, test my shiny aggregate function, and the masked pipeline, and many steps and aggresive learning rate 